## Import required libraries

In [19]:
import pandas as pd
import re
import requests
import numpy as np
import os.path

## Import raw data

In [2]:
df = pd.read_excel('https://query.data.world/s/slofesjnl4cdvy3jtv6qjyokuqhrjj')
df

,Date,Time,Location,Sub-location,Source,Notes
0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04
1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04
2,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01
3,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02
4,2021-05-30,1125h - 1200h,Ang Mo Supermarket (642 Hougang Avenue 8),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04
...,...,...,...,...,...,...
1311,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN
1312,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN
1313,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN
1314,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN


## Get longitude and latitude

In [3]:
request_url = "https://developers.onemap.sg/commonapi/search"

In [4]:
# exceptions
address_changes = {
    '37 & 39 Sultan Gate': '37 Sultan Gate',
    '1 Pasir Ris Drive 3': '1 Pasir Ris Drive',
    '08/09 Marine Parade Central': 'Marine Parade Central',
    '365 & 369 Balestier Road': '365 Balestier Road',
    '1/A Tampines Street 92': '1A Tampines Street 92',
    '1 HabourFront Walk': '1 HarbourFront Walk',
    '1 Jurong West Central 3': '1 Jurong West Central',
    'Block 69   Geylang   Bahru': '69 Geylang Bahru',
    '63-67 Temple Street': '63 Temple Street',
    '612/620 Geylang Lorong 38': '612 Geylang Lorong 38',
    '120 Pasir Ris Central Street 3': '120 Pasir Ris Central',
    '1 Imbiah Road': 'Imbiah Road',
    '163 Geylang Road Lorong 3': '163 Geylang Road',
    'Block 145 Teck Whye Avenue': '145 Teck Whye Avenue',
    '1 Harborfront Walk': '1 HarbourFront Walk',
    '803 King George’s Avenue': '803 King George Avenue',
    '863 Woodlands Street 62': '863 Woodlands',
    '8 Bishan Place': '9 Bishan Place',
    '389 Yung Sheng Road': 'Yung Sheng Road',
    '339 Yung Sheng Road': 'Yung Sheng Road'
}

In [5]:
for index, row in df.iterrows():
    result = re.search('(?s:.*)\((.*?)\)', row['Location'])
    location = result.group(1) if result else location
    location = address_changes.get(location) if location in address_changes else location
    
    # call request
    response = requests.get(request_url, params={
        'searchVal': location,
        'returnGeom': 'Y',
        'getAddrDetails': 'Y'
    })
    print('{}: {}'.format(index, location))
    result = response.json()['results'][0]
    df.at[index,'Longitude']=result['LONGITUDE']
    df.at[index,'Latitude']=result['LATITUDE']
    df.at[index,'Postal']=result['POSTAL']
df

0: 53 Pioneer Road
1: 511 Bukit Batok Street 2
2: 476D Upper Serangoon View
3: 21 Hougang Street 51
4: 642 Hougang Avenue 8
5: 3 New Bugis Street
6: 38 Beo Crescent
7: 10 Bayfront Avenue
8: 10 Bayfront Avenue
9: 308 Anchorvale Road
10: 33 Sengkang West Avenue
11: 476D Upper Serangoon View
12: 78 Redhill Lane
13: 111 North Bridge Road
14: 7 Pioneer Road North
15: 1 HarbourFront Walk
16: 750A Chai Chee Road
17: 763 Jurong West Street 75
18: 68 Orchard Road
19: 68 Orchard Road
20: 442 Orchard Road
21: 308 Anchorvale Road
22: 620 Hougang Avenue 8
23: 21 Hougang Street 51
24: 348 Bedok Road
25: 946 Margaret Drive
26: 455 Sengkang West Avenue
27: 308 Anchorvale Road
28: 33 Sengkang West Avenue
29: 33 Sengkang West Avenue
30: 308 Anchorvale Road
31: 71 Pioneer Road
32: 71 Pioneer Road
33: 455 Sengkang West Avenue
34: 417 Fernvale Link
35: 4000 Jalan Bukit Merah
36: 8 Marina Boulevard
37: 237 Serangoon Avenue 3
38: 298 Tiong Bahru Road
39: 50 Havelock Road
40: 348 Bedok Road
41: 809 French Roa

324: 208B New Upper Changi Road
325: 311 New Upper Changi Road
326: 201 Victoria Street
327: 10 Jurong East Street 12
328: 500 Lorong 6 Toa Payoh
329: 9 Bishan Place
330: 60 Paya Lebar Road
331: 200 Turf Club Road
332: 440 Bukit Batok West Avenue 8
333: 440 Bukit Batok West Avenue 8
334: 2 Choa Chu Kang Loop
335: 4 Bukit Batok Street 41
336: 444 Pasir Ris Drive 6
337: 1 Sengkang Square
338: 345 Jurong East Street 31
339: 51 Ang Mo Kio Avenue 3
340: 118 Rivervale Drive
341: 441 Pasir Ris Drive 6
342: 111 Somerset Road
343: 9 Tampines Street 32
344: 3 Gateway Drive
345: 118 Rivervale Drive
346: 1 Jelebu Road
347: 391 Orchard Rd
348: 118 Rivervale Drive
349: 437 Fernvale Road
350: 3 Gateway Drive
351: 6 Raffles Boulevard
352: 5 Changi Business Park Central 1
353: 823 Tampines Street 81
354: 5 Changi Business Park Central 1
355: 33 Sengkang West Avenue
356: 500 Dover Road
357: 10 Bayfront Avenue
358: 180 Kitchener Road
359: 68 Orchard Road
360: 1 Pasir Ris Central Street 3
361: 3 Gateway D

641: 51 Yishun Central 1
642: 20 Marsiling Lane
643: 53 Boat Quay
644: 1 Jurong West Central
645: 7 Maxwell Road
646: 17 Upper Boon Keng Road
647: 26A Kallang Place
648: 105 Hougang Avenue 1
649: 1382 Ang Mo Kio Avenue 1
650: 93 Stamford Road
651: 6 Raffles Boulevard
652: 22 Keong Saik Road
653: 3 New Bugis Street
654: 1 Keong Saik Road
655: 23 Serangoon Central
656: 23 Serangoon Central
657: 71 Sungei Kadut Drive
658: 273C Punggol Place
659: 50 Hougang Avenue 1
660: 1 Rochor Canal Road
661: 145 Marine Parade Road
662: 363 Joo Chiat Road
663: 30 Merchant Road
664: 8 Maju Avenue
665: 95 Syed Alwi Road
666: 5001 Beach Road
667: 83 Geylang Road
668: 10 Tebing Lane
669: 26A Kallang Place
670: 10 Tebing Lane
671: 3 Simei Street 6
672: 3 Simei Street 6
673: 1 Pasir Ris Close
674: 10 Tampines Central 1
675: 1 Pasir Ris Close
676: 1 Pasir Ris Close
677: 11 Bedok North Street 1
678: 60 Paya Lebar Road
679: 5 Straits View
680: 311 New Upper Changi Road
681: 10 Tampines Central 1
682: 4 Tampines 

978: 56 Jalan Kembangan
979: 80 Changi Boulevard
980: 65 Airport Boulevard
981: 1 Geylang Serai
982: 304 Orchard Road
983: 83 Punggol Central
984: 60 Nanyang Drive
985: 304 Orchard Road
986: 23 Serangoon Central
987: 118 Holland Avenue
988: 1 Pasir Ris Close
989: 60 Paya Lebar Road
990: 10 Kallang Road
991: 1 HarbourFront Walk
992: 612 Geylang Lorong 38
993: 304 Orchard Road
994: 18 Marina Gardens Drive
995: 1 Pasir Ris Central Street 3
996: 18 Marina Gardens Drive
997: 53 Ang Mo Kio Avenue 3
998: 1 HarbourFront Walk
999: 145 Syed Alwi Road
1000: 78 Airport Boulevard
1001: 304 Orchard Road
1002: 51 Old Airport Road
1003: 206 Bedok North Street 1
1004: 4A Jalan Batu
1005: 120 Pasir Ris Central
1006: 304 Orchard Road
1007: 1 Park Road
1008: 6 Eunos Crescent
1009: 1 Sengkang Square
1010: 435 Orchard Road
1011: 530 Ang Mo Kio Avenue 10
1012: 391A Orchard Road
1013: 4A Jalan Batu
1014: 260 Orchard Road
1015: 545 Orchard Road
1016: 290 Orchard Road
1017: 14 Scotts Road
1018: 3 Gateway Drive


1298: 9 Bishan Place
1299: 505 Jurong West St 52
1300: 2 Jurong East Street 21
1301: Yung Sheng Road
1302: 2 Jurong East Street 21
1303: 2 Jurong East Street 21
1304: 1 Jelebu Road
1305: 17 Petir Road
1306: 1 Jurong West Central 2
1307: 50 Jurong Gateway Road
1308: Yung Sheng Road
1309: 1 Jurong West Central 2
1310: 80 Marine Parade
1311: 1 Jurong West Central 2
1312: Yung Sheng Road
1313: 162 Ang Mo Kio Avenue 4
1314: 505 Jurong West St 52
1315: 100 Tras Street


,Date,Time,Location,Sub-location,Source,Notes,Longitude,Latitude,Postal
0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.657832233969,1.31133149338762,628505
1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.755832339467,1.34175978313366,659545
2,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01,103.903623796196,1.37791506138101,537476
3,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02,103.887847089168,1.37931783471773,538719
4,2021-05-30,1125h - 1200h,Ang Mo Supermarket (642 Hougang Avenue 8),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.879948293051,1.37169878879802,530642
...,...,...,...,...,...,...,...,...,...
1311,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.706685012654,1.33945200436318,NIL
1312,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.721386502056,1.33414175848059,NIL
1313,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.838900879239,1.37406230373928,560162
1314,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.718447549568,1.34964230218296,640505


In [6]:
df.to_csv('data/Covid-Public-SG-LngLat.csv')

## Getting planning areas

In [7]:
lng_lat_df = pd.read_csv('data/Covid-Public-SG-LngLat.csv')
lng_lat_df

,Unnamed: 0,Date,Time,Location,Sub-location,Source,Notes,Longitude,Latitude,Postal
0,0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.657832,1.311331,628505
1,1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.755832,1.341760,659545
2,2,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01,103.903624,1.377915,537476
3,3,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02,103.887847,1.379318,538719
4,4,2021-05-30,1125h - 1200h,Ang Mo Supermarket (642 Hougang Avenue 8),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.879948,1.371699,530642
...,...,...,...,...,...,...,...,...,...,...
1311,1311,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.706685,1.339452,NIL
1312,1312,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.721387,1.334142,NIL
1313,1313,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.838901,1.374062,560162
1314,1314,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.718448,1.349642,640505


In [8]:
# Obtained from https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore
planning_area_list = ['Ang Mo Kio', 'Bedok', 'Bishan', 'Boon Lay', 'Bukit Batok', 'Bukit Merah', 'Bukit Panjang', 'Bukit Timah', 'Central Water Catchment', 'Changi Airport', 'Changi Bay', 'Choa Chu Kang', 'Clementi', 'Downtown Core', 'Geylang', 'Hougang', 'Jurong East', 'Jurong West', 'Kallang', 'Lim Chu Kang', 'Mandai', 'Marina East', 'Marina South', 'Marine Parade', 'Museum', 'Newton', 'North-Eastern Islands', 'Novena', 'Orchard', 'Outram', 'Pasir Ris', 'Paya Lebar', 'Pioneer', 'Punggol', 'Queenstown', 'River Valley', 'Rochor', 'Seletar', 'Sembawang', 'Sengkang', 'Serangoon', 'Simpang', 'Singapore River', 'Southern Islands', 'Straits View', 'Sungei Kadut', 'Tampines', 'Tanglin', 'Tengah', 'Toa Payoh', 'Tuas', 'Western Islands', 'Western Water Catchment', 'Woodlands', 'Yishun']

In [32]:
planning_area_df = pd.DataFrame({'Area': planning_area_list})
planning_area_df.head()

,Area
0,Ang Mo Kio
1,Bedok
2,Bishan
3,Boon Lay
4,Bukit Batok


In [35]:
planning_area_path = 'data/SG-Planning-Area.csv'

if os.path.isfile(planning_area_path):
    planning_area_df = pd.read_csv(planning_area_path)
else:
    for index, row in planning_area_df.iterrows():
        response = requests.get(request_url, params={
            'searchVal': row['Area'],
            'returnGeom': 'Y',
            'getAddrDetails': 'N'
        })
#         print('{}: {}'.format(index, row['Area']))
        if len(response.json()['results']) <= 0:
            continue
        result = response.json()['results'][0]
        planning_area_df.at[index,'Longitude']=result['LONGITUDE']
        planning_area_df.at[index,'Latitude']=result['LATITUDE']

    planning_area_df = planning_area_df.dropna()

    planning_area_df.to_csv(planning_area_path, index=False)

planning_area_df.head()

,Area,Longitude,Latitude
0,Ang Mo Kio,103.837993,1.368750
1,Bedok,103.953607,1.325015
2,Bishan,103.852048,1.347522
3,Boon Lay,103.705845,1.338493
4,Bukit Batok,103.753493,1.343794


In [11]:
baseline = planning_area_df.iloc[0]

def find_place(lng, lat):
    result = baseline['Area']
    
    baseline_a = np.array((float(baseline['Longitude']), float(baseline['Latitude'])))
    b = np.array((lng, lat))
    
    minimum_distance = np.linalg.norm(baseline_a-b)
    
    for index, row in planning_area_df.iterrows():
        a = np.array((float(row['Longitude']), float(row['Latitude'])))
        distance = np.linalg.norm(a-b)
        if distance < minimum_distance:
            minimum_distance = distance
            result = row['Area']
    
    return result

In [12]:
for index, row in lng_lat_df.iterrows():
    lng_lat_df.at[index,'Area']=find_place(row['Longitude'], row['Latitude'])
lng_lat_df

,Unnamed: 0,Date,Time,Location,Sub-location,Source,Notes,Longitude,Latitude,Postal,Area
0,0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.657832,1.311331,628505,Tuas
1,1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.755832,1.341760,659545,Bukit Batok
2,2,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01,103.903624,1.377915,537476,Sengkang
3,3,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02,103.887847,1.379318,538719,Sengkang
4,4,2021-05-30,1125h - 1200h,Ang Mo Supermarket (642 Hougang Avenue 8),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.879948,1.371699,530642,Seletar
...,...,...,...,...,...,...,...,...,...,...,...
1311,1311,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.706685,1.339452,NIL,Boon Lay
1312,1312,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.721387,1.334142,NIL,Boon Lay
1313,1313,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.838901,1.374062,560162,Ang Mo Kio
1314,1314,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.718448,1.349642,640505,Boon Lay


In [13]:
lng_lat_df.to_csv('data/Covid-Public-SG-LngLat-Area.csv')

## Using postal code to get region 
Does not work because a lot of NIL postal codes

In [14]:
# sg_postal_df = pd.read_csv('data/SG-postal.csv')
# sg_postal_df.columns = sg_postal_df.columns.str.strip()
# intermediate_sg_postal_df = pd.concat([sg_postal_df['Postal Sector (1st 2 digits of postal codes)'].str.split(',', expand=True), 
#                  sg_postal_df['General Location'], 
#                  sg_postal_df['Postal District']], 
#                 axis=1)
# intermediate_sg_postal_df
# lookup_sg_postal_df = pd.melt(intermediate_sg_postal_df,id_vars=['Postal District','General Location'],value_name='Postal Sector').dropna().drop(columns=['variable'])


# lookup_sg_postal_df.sort_values("Postal Sector")

In [15]:
# df = df.replace("NIL", None)
# df['Postal-sector'] = df['Postal'].astype(str).str[:2]
# result_df = pd.merge(df, lookup_sg_postal_df, left_on="Postal-sector", right_on="Postal Sector", how="left")
# result_df

In [16]:
# result_df.to_csv('dtata/Covid-Public-SG-LngLat-Postal.csv')

In [17]:
# lng_lat_df = pd.read_csv('data/Covid-Public-SG-LngLat.csv')
# lng_lat_df['Postal'][lng_lat_df['Postal'] == "NIL"].count()